In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install tensorflow

In [ ]:
# Import necessary libraries

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator


from numpy.random import seed
import tensorflow
seed(1)
tensorflow.random.set_seed(2)


import cv2
import matplotlib.pyplot as plt
import random

from glob import glob
from numpy import floor


# Plotting sample images from the datastes

def plot_samples(alphabet):
    print("Samples images for alphabet " + alphabet)
    base_path = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/'
    img_path = base_path + alphabet + '/**'
    path_contents = glob(img_path)
    
    plt.figure(figsize=(16,16))
    imgs = random.sample(path_contents, 4)
    plt.subplot(151)
    plt.imshow(cv2.imread(imgs[0]))
    plt.subplot(152)
    plt.imshow(cv2.imread(imgs[1]))
    plt.subplot(153)
    plt.imshow(cv2.imread(imgs[2]))
    plt.subplot(154)
    plt.imshow(cv2.imread(imgs[3]))
    return

plot_samples('A')

In [ ]:
# Reading datasets and performing data augmentation

train_dir = "../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
target_size = (64, 64)
target_dims = (64, 64, 3) # RGB - 3, BW - 2
no_classes = 29
val_split = 0.1
batch_size = 64
 
data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True, 
                                    validation_split=val_split)

train_generator = data_augmentor.flow_from_directory(train_dir, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(train_dir, target_size=target_size, batch_size=batch_size, subset="validation")

In [ ]:
model = Sequential()
conv2d_1 = Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=target_dims)
model.add(conv2d_1)
conv2d_2 = Conv2D(64, kernel_size=4, strides=2, activation='relu')
model.add(conv2d_2)
model.add(Dropout(0.5))
conv2d_3 = Conv2D(128, kernel_size=4, strides=1, activation='relu')
model.add(conv2d_3)
conv2d_4 = Conv2D(128, kernel_size=4, strides=2, activation='relu')
model.add(conv2d_4)
model.add(Dropout(0.5))
conv2d_5 = Conv2D(256, kernel_size=4, strides=1, activation='relu')
model.add(conv2d_5)
conv2d_6 = Conv2D(256, kernel_size=4, strides=2, activation='relu')
model.add(conv2d_6)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# Summary of the model
model.summary()

In [ ]:
# Fitting the model

model.fit_generator(train_generator, epochs=5, validation_data=val_generator)